In [22]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
t = tf.ones([1,3,2,4])
print(t)

tf.Tensor(
[[[[1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]]]], shape=(1, 3, 2, 4), dtype=float32)


In [3]:
a = tf.constant([1, 1, 1])
b = tf.constant([2])
print(a * b)

tf.Tensor([2 2 2], shape=(3,), dtype=int32)


In [24]:
w = tf.Variable(tf.random.normal((3, 2)), name = 'weights')
x = [[1., 1., 1.]]
b = tf.Variable(tf.zeros(2, dtype = tf.float32), name='biases')

with tf.GradientTape(persistent=True) as tape:
    y = tf.matmul(x, w) + b
y= tf.nn.sigmoid(y)  

In [25]:
print(y)

tf.Tensor([[0.8336704 0.5998972]], shape=(1, 2), dtype=float32)


In [26]:
[v.name for v in tape.watched_variables()]

['weights:0', 'biases:0']

In [27]:
gradients = tape.gradient(y, [w, b])
print(gradients)

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.13866408, 0.24002054],
       [0.13866408, 0.24002054],
       [0.13866408, 0.24002054]], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.13866408, 0.24002054], dtype=float32)>]


In [36]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, output_features, **kwargs):
        super().__init__(**kwargs)
        self.output_features = output_features
    
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal((input_shape[-1], self.output_features)), name = 'weights')
        self.b = tf.Variable(tf.zeros(self.output_features, dtype = tf.float32), name='biases')
    
    def call(self, inputs):
        return tf.nn.sigmoid(tf.matmul(inputs, self.w) + self.b)

In [37]:
my_dense = MyDense(output_features = 3)

In [38]:
my_dense(tf.constant([[1., 1., 1.], [2., 2., 2.]]))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.65940213, 0.33986932, 0.6029423 ],
       [0.7893917 , 0.20953123, 0.6975124 ]], dtype=float32)>

In [43]:
class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
        self.layer_1 = MyDense(output_features = 3)
        self.layer_2 = MyDense(output_features = 1)
        
    def call(self, input):
        return self.layer_1(self.layer_2(input))

In [44]:
my_custom_model = MyModel()
my_custom_model(tf.constant([[1., 1., 1.], [2., 2., 2.]]))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.73259264, 0.51657116, 0.31847566],
       [0.7387214 , 0.51708895, 0.31333414]], dtype=float32)>

In [45]:
my_custom_model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_3 (MyDense)         multiple                  6         
_________________________________________________________________
my_dense_4 (MyDense)         multiple                  4         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [59]:
@tf.custom_gradient
def f(x):
    def grad(upstream):
        return upstream * 2 * x
    return x**2, grad

In [60]:
tf.compat.v1.disable_eager_execution()
x = tf.constant(100.)
y = f(x)
dy_dx= tf.gradients(y, x)
print(dy_dx)

[<tf.Tensor 'gradients_7/IdentityN_5_grad/mul_1:0' shape=() dtype=float32>]
